In [1]:
import numpy as np
import pandas as pd
import os, glob
import nibabel as nib
import math
from scipy.ndimage import binary_dilation
from skimage import segmentation, morphology

In [2]:
# Collects all the segmentation maps (make sure this segmentation map is binary segmentation of whole tumor)
seg_dir = 'segmentation map folder directory'
segm_dir = sorted(glob.glob(os.path.join(seg_dir, "*Seg*.nii.gz")))

# Collects appropriate Rec ROI(dilated)
roi_dir = 'C:/Users/Sunwoo/Documents/Research/DL/Patch/Data/Dilated_rec'
rec_dilated_dir = sorted(glob.glob(os.path.join(roi_dir, "*Rec*.nii.gz"))) 

# Collects FLAIR image from baseline directory
baseline_dir = 'C:/Users/Sunwoo/Documents/Research/DL/Patch/Data/Baseline'
flair_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_flair*.nii.gz"))) 

In [3]:
# Load nifti files into numpy array
def LoadingImage(dir):
    
    nifti_image = nib.load(dir)
    image = np.asarray(nifti_image.dataobj)
    header = nifti_image.header
    imgaffine = nifti_image.affine
    
    return image, header, imgaffine

In [4]:
# Get the unique name of subject
def getFilename(full_dir):
    _,filename = full_dir.split('\\')
    print(filename)
    subject, _= filename.split('_')
    return str(subject)

In [ ]:
# Set uncertain area distance from Recurrence ROI and Tumor Core
uncertain_dist = 7

# Itereate each subject
for i in range(len(segm_dir)):

    # call all the images
    seg_img, seg_hdr, seg_imgaffine = LoadingImage(segm_dir[i])
    # call rec ROI (240, 240, 155)
    rec_img, rec_hdr, rec_imgaffine = LoadingImage(rec_dilated_dir[i])
    # call flair image (240, 240, 155)
    flair_img, flair_hdr, flair_imgaffine = LoadingImage(flair_dir[i])

    # Initialize an empty array to store the aggregated boundaries
    aggregated_boundaries = np.zeros_like(seg_img)

    # Create the initial boundary using the original image
    initial_boundary = segmentation.find_boundaries(seg_img, mode='inner')
    aggregated_boundaries = np.logical_or(aggregated_boundaries, initial_boundary)

    # Erode the labels and aggregate the boundaries
    eroded_labels = seg_img.copy()
    for k in range(6):  # Adjust this range to control the thickness of the boundary(here we have 7 voxel thick)
        eroded_labels = morphology.binary_erosion(eroded_labels)
        inner_boundaries = segmentation.find_boundaries(eroded_labels, mode='inner')
        aggregated_boundaries = np.logical_or(aggregated_boundaries, inner_boundaries)

    # Convert boolean mask to int for visualization
    boundary_image = aggregated_boundaries.astype(int)
    
    ################################# REMOVING VOXELS NEAR REC ROI AND TUMOR CORE ###################################
    # make sure boundary image only contains edema + no tumor included
    boundary_checkEdema = np.where(seg_img != 2, 0, boundary_image)
    boundary_checkTumor = np.where((seg_img == 4) | (seg_img == 1), 0, boundary_checkEdema)

    #collect coordinates of the edema from seg
    edema_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if seg_img[x,y,z] == 2]

    #collect coordinates of the tumor from seg
    tumor_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if seg_img[x,y,z] == 4 or seg_img[x,y,z] == 1]

    # collect coordinates of the rec
    rec_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if rec_img[x,y,z] > 0.9 and rec_img[x,y,z] < 1.1]

    print(len(rec_roi_coord))

    # collect coordinates of the rec
    boundary_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if boundary_checkTumor[x,y,z] == 1]

    print(len(boundary_roi_coord))

    #zeroing out 7mm from tumor and rec
    for boundary_voxel in boundary_roi_coord:
        for tumor_voxel in tumor_roi_coord:
            distance_edema_rec = math.dist(boundary_voxel,tumor_voxel)
            if distance_edema_rec <= uncertain_dist:
                x = boundary_voxel[0]
                y = boundary_voxel[1]
                z = boundary_voxel[2]
                boundary_checkTumor[x,y,z] = np.int16(0)
    
    # collect coordinates of the rec
    boundary_roi_coord2=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if boundary_checkTumor[x,y,z] == 1]

    #zeroing out 7mm from tumor and rec
    for boundary_voxel in boundary_roi_coord2:
        for rec_voxel in rec_roi_coord:
            distance_edema_rec = math.dist(boundary_voxel,rec_voxel)
            if distance_edema_rec <= uncertain_dist:
                x = boundary_voxel[0]
                y = boundary_voxel[1]
                z = boundary_voxel[2]
                boundary_checkTumor[x,y,z] = np.int16(0)
    
    ################################################## CHECK FLAIR VALUES ##########################################
    
    mask = seg_img == 2
    cutoff = np.percentile(flair_img[mask].flatten(), 20)  # Get the 20th percentile (bottom 20%)

    # Check flair is less than bottom 20% (cutoff)
    boundary_update = np.where(flair_img < cutoff, 0, boundary_checkTumor)

    # filename
    filename = getFilename(segm_dir[i])

    #save the distance map
    nifti_prob = nib.Nifti1Image(boundary_update, seg_imgaffine, header = seg_hdr)
    dir_path = 'Save created Non-Infiltrative ROI in certain directory'
    nib.save(nifti_prob, dir_path + 'Name your files')